In [1]:
import torch
import torchvision
import torchvision.models as models
from PIL import Image, ImageFile
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import gc; gc.enable()
import pandas as pd
from torch.utils.data import Dataset
import tensorflow as tf
from tensorflow import keras
from torchvision import transforms

In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
# keras imports
from keras.models import Model
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Input
# other imports
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score
import numpy as np
import glob
import h5py
import json
import datetime
import time

Using TensorFlow backend.


In [3]:
import efficientnet.keras as efn 

f7 = efn.EfficientNetB7(include_top = False, weights='imagenet')

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


In [4]:
from tensorflow.keras.callbacks import Callback
# Specify title of our final model
SAVED_MODEL_NAME = 'effnet_modelB7.h5'
def get_preds_and_labels(model, generator):
    """
    Get predictions and labels from the generator
    """
    preds = []
    labels = []
    for _ in range(int(np.ceil(generator.samples / batch_size))):
        x, y = next(generator)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel(), np.concatenate(labels).ravel()

class Metrics(Callback):
    """
    A custom Keras callback for saving the best model
    according to the Quadratic Weighted Kappa (QWK) metric
    """
    def on_train_begin(self, logs={}):
        """
        Initialize list of QWK scores on validation data
        """
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        """
        Gets QWK score on the validation data
        """
        # Get predictions and convert to integers
        y_pred, labels = get_preds_and_labels(model, val_generator)
        y_pred = np.rint(y_pred).astype(np.uint8).clip(0, 4)
        # We can use sklearns implementation of QWK straight out of the box
        # as long as we specify weights as 'quadratic'
        _val_kappa = cohen_kappa_score(labels, y_pred, weights='quadratic')
        self.val_kappas.append(_val_kappa)
        print(f"val_kappa: {round(_val_kappa, 4)}")
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save(SAVED_MODEL_NAME)
        return

In [5]:
train_dir = '/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/data/'
img_dir = train_dir + 'train/'

In [6]:
train_df = pd.read_csv(train_dir+'dr15labels_2.csv')
train_df_2 = pd.read_csv('/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/drlabels.csv')

In [9]:
set_trainable = False
for layer in f7.layers:
    if 'block3' in layer.name:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
from keras import layers
def build_model():
    """
    A custom implementation of EfficientNetB7
    for the APTOS 2019 competition
    """
    model = Sequential()
    model.add(f7)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512))
    model.add(layers.Dense(5, activation='elu'))
#     model.add(layers.Dense(1, activation="linear"))
#     print(model.summary())
    return model

# Initialize model
model = build_model()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import optimizers
datagen = keras.preprocessing.image.ImageDataGenerator(validation_split = 0.15)
batch_size = 32
train_steps_per_epoch = len(subs)//batch_size
adm = optimizers.Adam(lr = 0.0001)

train_generator = datagen.flow_from_dataframe(subs, 
                                              x_col = 'id_code', 
                                              y_col = 'diagnosis',
                                              target_size = (224,224),
                                              batch_size = batch_size,
                                              class_mode ='categorical',
                                              subset = 'training') 
val_generator = datagen.flow_from_dataframe(subs, 
                                            x_col='id_code', 
                                            y_col='diagnosis',
                                            target_size=(224, 224),
                                            batch_size=batch_size,
                                            class_mode='categorical',
                                            subset = 'validation')
# For tracking Quadratic Weighted Kappa score
kappa_metrics = Metrics()
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=10)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=3, 
                        verbose=1, 
                        mode='auto', 
                        epsilon=0.0001)

model.compile(optimizer = adm, loss = 'categorical_crossentropy')
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch, 
                              validation_data=val_generator,
                              validation_steps = val_generator.samples // batch_size,
                              epochs=150, verbose=1, callbacks=[kappa_metrics, es, rlr])

Found 4248 validated image filenames belonging to 5 classes.
Found 749 validated image filenames belonging to 5 classes.


/Users/flatironschool/.local/lib/python3.6/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 2167 invalid image filename(s) in x_col="id_code". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Epoch 1/150
 65/223 [=======>......................] - ETA: 4:26:45 - loss: 6.2830

In [ ]:
plt.plot(history.history['loss'])
plt.savefig('loss_f7')

In [ ]:
model_json = model.to_json()
with open("f7.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("f7.h5")
print("Saved model to disk")